<a href="https://colab.research.google.com/github/klarcika/TZ/blob/main/Naloga9_Pridobivanje_podatkov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraping data from Tripadvisor.
Finding out which restaurants are rated the best and why


In [ ]:
!pip install Scrapy

In [ ]:
import scrapy
import re
import numpy as np  # Import numpy for nan values

class Spider(scrapy.Spider):
    name = "properties"
    allowed_domains = ["sloveniaestates.com"]
    start_urls = ["https://www.sloveniaestates.com/properties/view-all-properties/"]
    custom_settings = {
        'FEEDS': {
            'properties.csv': {
                'format': 'csv',
                'overwrite': True
            }
        }
    }

    def parse(self, res):
        vsa_dela = res.css(".masonryitem.clearfix")

        for delo in vsa_dela:
            d = {
                "naziv": delo.css("h2::text").extract_first(default=np.nan),
                "type": delo.css("ul li.clearfix.right::text").extract_first(default=np.nan),
                "price": delo.css("ul li.clearfix::text").extract()[1].strip() if len(delo.css("ul li.clearfix::text")) > 1 else np.nan,
                "size": delo.css("ul li.clearfix.right::text").extract_first(default=np.nan),
                "size of land": delo.css("ul li.clearfix::text").extract()[1].strip() if len(delo.css("ul li.clearfix::text")) > 1 else np.nan,
                "bedrooms": delo.css("ul li.clearfix.right::text").extract_first(default=np.nan),
            }
            yield d

        naslednje_strani = res.css("div.filterBox.clearfix div.paging::text").extract()

        for naslednja in naslednje_strani:
            if int(naslednja) < 5:
                url_str = f"https://www.sloveniaestates.com/properties/view-all-properties/?paging={naslednja}"
                yield scrapy.Request(url_str, callback=self.parse)


In [ ]:
from scrapy.crawler import CrawlerProcess

proces= CrawlerProcess()
proces.crawl(spider)

INFO:scrapy.utils.log:Scrapy 2.11.0 started (bot: scrapybot)
2023-12-05 21:46:38 [scrapy.utils.log] INFO: Scrapy 2.11.0 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-5.15.120+-x86_64-with-glibc2.35
2023-12-05 21:46:38 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 23.3.0 (OpenSSL 3.1.4 24 Oct 2023), cryptography 41.0.7, Platform Linux-5.15.120+-x86_64-with-glibc2.35
INFO:scrapy.addons:Enabled addons:
[]
2023-12-05 21:46:38 [scrapy.addons] INFO: Enabled addons:
[]
DEBUG:scrapy.utils.log:Using reactor: twisted.internet.epollreactor.EPollReactor
2023-12-05 21:46:38 [scrapy.utils.log] DEBUG: Using reactor: